In [3]:
import numpy as np
import os.path
import sys
from keras.models import Sequential, load_model
from keras.layers import Convolution2D, MaxPooling2D, Convolution3D, MaxPooling3D
#from keras.layers.convolutional import Conv2D, Conv3D
#from keras.layers.pooling import MaxPooling2D, MaxPooling3D
from keras.layers.core import Flatten, Dense, Lambda
from keras import backend as K
from keras.optimizers import SGD, Adagrad, Adadelta
from keras.callbacks import CSVLogger

airports = ['EIDW', 'EGLL', 'LFPG', 'LFBO', 'EGPH', 'EHAM', 'EBBR', 'LEMD', 'LEBL', 'LPPT', 'LIRF',
            'LIMC', 'LSZH', 'EDDM', 'EDFH', 'EDDT', 'EKCH', 'ENGM', 'ESSA', 'EFHK', 'LOWW']

def get_rains(code):
    arr = np.load("/home/guilletrejo/DeepWeather-master/short/rain_shorter.npy")
    idx = airports.index(code)
    return arr[:, idx].astype(np.int32)

def get_era_full(param, level):
    arr = np.load("/home/guilletrejo/DeepWeather-master/short/{}{}_shorter_uint8.npy".format(param, level))
    return arr.astype(np.float32) / 256.

def train_model(airport):
    # Import data
    params = ["z", "z", "z"]
    levels = [500, 700, 1000]

    in1_var = get_era_full(params[0], levels[0])
    in2_var = get_era_full(params[1], levels[1])
    in3_var = get_era_full(params[2], levels[2])

    print("Shape in1 = " + str(in1_var.shape))
    print("Shape in2 = " + str(in2_var.shape))
    print("Shape in3 = " + str(in3_var.shape))
    
    # Agrega un eje mas interno a los arreglos input
    in1_var_ex = np.expand_dims(in1_var, axis=3)
    in2_var_ex = np.expand_dims(in2_var, axis=3)
    in3_var_ex = np.expand_dims(in3_var, axis=3)

    print("Shape in1_ex = " + str(in1_var_ex.shape))
    print("Shape in2_ex = " + str(in2_var_ex.shape))
    print("Shape in3_ex = " + str(in3_var_ex.shape))

    # Concatena los 3 input de 4 dimensiones y shape 140,80,120,1 en un solo input 
    # de 4 dimensiones con shape 140,80,120,3
    x = np.concatenate((in1_var_ex, in2_var_ex, in3_var_ex), axis=3)
    print(x.shape)
    
    X = np.zeros((93, 80, 120, 8, 3))
    for i in range(93):
        X[i,:,:,:,:] = np.rollaxis(x[i:i+8, :, :, :],0,3)

    Y = get_rains(airport)[7:]
    b = np.zeros((Y.shape[0], 2))
    b[np.arange(Y.shape[0]), Y] = 1

    model = None
    if os.path.isfile('model_3d_{}.h5'.format(airport)):
        model = load_model('model_3d_{}.h5'.format(airport))
    else:
        model = Sequential()
        model.add(Convolution3D(128, (3, 3, 3), padding='same', activation='relu', name='block1_conv1', input_shape=(80,120,8,3)))
        model.add(MaxPooling3D((2, 2, 2), strides=(2, 2, 2)))

        model.add(Convolution3D(256, (3, 3, 3), padding='same', activation='relu', name='block2_conv1'))
        model.add(MaxPooling3D((2, 2, 2), strides=(2, 2, 2)))

        model.add(Flatten())
        model.add(Dense(2, activation = 'softmax', name='final_fully_connected'))

        adagrad = Adagrad(lr=0.0002)
        model.compile(loss='categorical_crossentropy', optimizer = adagrad, metrics=['accuracy'])

    csv_logger = CSVLogger('{}.log'.format(airport))
    model.fit(X, b, batch_size=20, epochs=5, verbose=1, validation_split=0.2, callbacks=[csv_logger])

    model.save('model_3d_{}.h5'.format(airport))


In [4]:
if __name__ == '__main__':
    airport = 'EIDW'
    model3d = train_model(airport)

Shape in1 = (100, 80, 120)
Shape in2 = (100, 80, 120)
Shape in3 = (100, 80, 120)
Shape in1_ex = (100, 80, 120, 1)
Shape in2_ex = (100, 80, 120, 1)
Shape in3_ex = (100, 80, 120, 1)
(100, 80, 120, 3)
Train on 74 samples, validate on 19 samples
Epoch 1/5
74/74 [==============================] - 179s 2s/step - loss: 1.1228 - acc: 0.6216 - val_loss: 0.6655 - val_acc: 0.6316
Epoch 2/5
74/74 [==============================] - 177s 2s/step - loss: 0.6609 - acc: 0.5676 - val_loss: 0.6713 - val_acc: 0.6316
Epoch 3/5
74/74 [==============================] - 177s 2s/step - loss: 0.6875 - acc: 0.6216 - val_loss: 0

In [5]:
import numpy as np
# Estos archivos son los datos, los samples
matriz500 = np.load("/home/guilletrejo/DeepWeather-master/z500_uint8.npy")
matriz700 = np.load("/home/guilletrejo/DeepWeather-master/z700_uint8.npy")
matriz1000 = np.load("/home/guilletrejo/DeepWeather-master/z1000_uint8.npy")


In [6]:
print(matriz500.shape)
print(matriz700.shape)
print(matriz1000.shape)


(13148, 80, 120)
(13148, 80, 120)
(13148, 80, 120)


In [7]:
mat500 = matriz500[:1348][:][:]
mat700 = matriz700[:1348][:][:]
mat1000 = matriz1000[:1348][:][:]
print(mat500.shape)
print(mat700.shape)
print(mat1000.shape)


(1348, 80, 120)
(1348, 80, 120)
(1348, 80, 120)


In [12]:
X1 = np.load("/home/guilletrejo/DeepWeather-master/short/z500_shorter_uint8.npy")
X2 = np.load("/home/guilletrejo/DeepWeather-master/short/z700_shorter_uint8.npy")
X3 = np.load("/home/guilletrejo/DeepWeather-master/short/z1000_shorter_uint8.npy")

In [13]:
X1_ex = np.expand_dims(X1,axis=3)
X2_ex = np.expand_dims(X2,axis=3)
X3_ex = np.expand_dims(X3,axis=3)
Xall = np.concatenate((X1_ex,X2_ex,X3_ex),axis=3)
print(Xall.shape)

(100, 80, 120, 3)


In [26]:
xall = np.zeros((93, 80, 120, 8, 3))
for i in range(93):
    xall[i,:,:,:,:] = np.rollaxis(Xall[i:i+8, :, :, :],0,3)

In [29]:
def get_rains(code):
    arr = np.load("/home/guilletrejo/DeepWeather-master/short/rain_shorter.npy")
    idx = airports.index(code)
    return arr[:, idx].astype(np.int32)

y = get_rains('EIDW')
y = y[:93]
print(y.shape)
b = np.zeros((y.shape[0], 2))
b[np.arange(y.shape[0]), y] = 1
print(y.shape)
print(b.shape)



model1 = load_model('model_3d_EIDW.h5')
print(model1.evaluate(xall,b))
prediction = model1.predict(xall,verbose=1)

(93,)
(93,)
(93, 2)
93/93 [==============================] - 59s 630ms/step
[10.225458545069541, 0.36559139768923482]
93/93 [==============================] - 33s 358ms/step


In [31]:
print(b.shape)
print(prediction)
print(y)

(93, 2)
[[  6.20043561e-22   1.00000000e+00]
 [  9.22418363e-22   1.00000000e+00]
 [  2.11672712e-21   1.00000000e+00]
 [  2.40667715e-21   1.00000000e+00]
 [  5.84155110e-21   1.00000000e+00]
 [  1.13943507e-20   1.00000000e+00]
 [  1.25180659e-20   1.00000000e+00]
 [  1.20379094e-20   1.00000000e+00]
 [  1.06064911e-20   1.00000000e+00]
 [  1.37505028e-20   1.00000000e+00]
 [  2.60551779e-20   1.00000000e+00]
 [  6.69865206e-20   1.00000000e+00]
 [  1.55330018e-19   1.00000000e+00]
 [  5.70216852e-19   1.00000000e+00]
 [  7.67094058e-19   1.00000000e+00]
 [  1.48661455e-19   1.00000000e+00]
 [  6.76175399e-21   1.00000000e+00]
 [  2.64723267e-22   1.00000000e+00]
 [  3.01820896e-23   1.00000000e+00]
 [  3.50710519e-23   1.00000000e+00]
 [  2.97796481e-22   1.00000000e+00]
 [  7.56619279e-21   1.00000000e+00]
 [  7.82196334e-20   1.00000000e+00]
 [  2.35199656e-19   1.00000000e+00]
 [  2.07077377e-19   1.00000000e+00]
 [  8.07694305e-20   1.00000000e+00]
 [  5.95406180e-21   1.0000000

In [30]:
rain_short = rain[:1348][:][:]
print(rain_short.shape)
print(rain[:,0:2])

(1348, 21)
[[ True False]
 [ True  True]
 [ True False]
 ..., 
 [ True  True]
 [ True False]
 [False  True]]


In [35]:
np.save("/home/guilletrejo/DeepWeather-master/short/rain_short.npy",rain_short)

In [111]:
arreglo = np.zeros((2,3,2))
for box in arreglo:
    box[0][0] = 1
print(arreglo)

[[[ 1.  0.]
  [ 0.  0.]
  [ 0.  0.]]

 [[ 1.  0.]
  [ 0.  0.]
  [ 0.  0.]]]


In [107]:
arreglo2 = np.zeros((2,3,2))
for box in arreglo2:
    box[0][0] = 2.
    box[0][1] = 3.
print(arreglo2)

[[[ 2.  3.]
  [ 0.  0.]
  [ 0.  0.]]

 [[ 2.  3.]
  [ 0.  0.]
  [ 0.  0.]]]


In [115]:
arreglo1_ex = np.expand_dims(arreglo,axis=3)
arreglo2_ex = np.expand_dims(arreglo2,axis=3)
print(arreglo1_ex.shape)
print(arreglo2_ex.shape)
print(arreglo1_ex)
print(arreglo2_ex)
print(np.concatenate((arreglo1_ex,arreglo2_ex),axis=3))

(2, 3, 2, 1)
(2, 3, 2, 1)
[[[[ 1.]
   [ 0.]]

  [[ 0.]
   [ 0.]]

  [[ 0.]
   [ 0.]]]


 [[[ 1.]
   [ 0.]]

  [[ 0.]
   [ 0.]]

  [[ 0.]
   [ 0.]]]]
[[[[ 2.]
   [ 3.]]

  [[ 0.]
   [ 0.]]

  [[ 0.]
   [ 0.]]]


 [[[ 2.]
   [ 3.]]

  [[ 0.]
   [ 0.]]

  [[ 0.]
   [ 0.]]]]
[[[[ 1.  2.]
   [ 0.  3.]]

  [[ 0.  0.]
   [ 0.  0.]]

  [[ 0.  0.]
   [ 0.  0.]]]


 [[[ 1.  2.]
   [ 0.  3.]]

  [[ 0.  0.]
   [ 0.  0.]]

  [[ 0.  0.]
   [ 0.  0.]]]]
